<a href="https://colab.research.google.com/github/alwanfail/Capstone-Bangkit/blob/main/keras_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-ocr
!pip install --force-reinstall -v "tensorflow==2.15.1"
!pip install --force-reinstall -v "keras==2.15.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.6 MB/s eta 0:00:00
Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for tensorflow==2.15.1 from https://files.pythonhosted.org/packages/4f/42/433c0c64c5d3b8bee696cde2006d15f03f0504c2f746d49f38e32e52e239/tensorflow-2.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from http

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for keras==2.15.0 from https://files.pythonhosted.org/packages/fc/a7/0d4490de967a67f68a538cc9cdb259bff971c4b5787f7765dc7c8f118f71/keras-2.15.0-py3-none-any.whl.metadata
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Removing file or directory /usr/local/lib/python3.10/dist-packages/keras-2.15.0.dist-info/
      Removing file or directory /usr/local/lib/python3.10/dist-packages/keras/
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.1 which is incompati

In [3]:
import keras_ocr
import os
import cv2
import numpy as np
import json

In [4]:
train_dir = '/content/drive/My Drive/capstone/train'
val_dir = '/content/drive/My Drive/capstone/val'

In [5]:
def preprocess_images(image_dir, target_size=(640, 640)):
    images = []
    image_paths = []
    for file_name in os.listdir(image_dir):
        file_path = os.path.join(image_dir, file_name)
        try:
            # Baca gambar
            image = cv2.imread(file_path)
            if image is not None:
                # Resize gambar ke ukuran target
                image = cv2.resize(image, target_size)
                images.append(image)
                image_paths.append(file_path)
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    return np.array(images), image_paths


In [6]:
train_images, train_paths = preprocess_images(train_dir)
val_images, val_paths = preprocess_images(val_dir)

In [7]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.15.1
2.15.0


# **Deteksi Teks**

In [8]:
pipeline = keras_ocr.pipeline.Pipeline()

def detect_text(images, pipeline):
    predictions = []
    for image in images:
        prediction = pipeline.recognize([image])[0]
        predictions.append(prediction)
    return predictions

train_texts = detect_text(train_images, pipeline)
val_texts = detect_text(val_images, pipeline)

Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5
4/4 [==============================] - 0s 59ms/step


# **save extracted file**

In [11]:
def save_ocr_results(file_path, predictions):
    results = []
    for prediction in predictions:
        image_results = []
        for text, box in prediction:
            image_results.append({
                'text': text,
                'box': box.tolist()  # Konversi array ke list agar JSON-compatible
            })
        results.append(image_results)

    with open(file_path, 'w') as file:
        json.dump(results, file, indent=4)

In [12]:
save_ocr_results('/content/drive/MyDrive/train_ocr_results.json', train_texts)
save_ocr_results('/content/drive/MyDrive/val_ocr_results.json', val_texts)

Exception ignored in: <function _xla_gc_callback at 0x7d9d5f220ee0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
